<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0913e4c9c1cd5470d05113b78263fb84ff5de1bf8fb541d669cef94b18d31bb9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 10:21:23
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -35.43 K (-0.25%)
Current PnL: -22.93 L (-15.03%)
CY Booked + Current PnL: -9.04 L (-5.92%)
-------------------
Total profit:  1.60 L
Total loss:  -24.53 L
-------------------
Total Booked + Current PnL: 17.94 L (14.28%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.72%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.34 L (61.84%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,0.63,-13.69,15.97,0.09,13890.0,-13800.0,86977.0,101.92,52.0,M-SC,4.28,253.0,-0.99,0.61,14.27,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,0.57,-8.47,9.28,0.02,14826.0,-14791.0,159768.0,143.86,72.0,M-SC,5.48,234.0,-1.00,1.12,38.97,OX40N,NTT,PAINTS
43,ITC,452.0,-0.29,-0.64,11.15,10.44,22172.0,-1286.0,198852.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.89,X40,NTT,FMCG
83,VOLTAS,1530.0,-1.13,7.96,10.87,19.69,22501.0,15258.0,207000.0,-2.60,51.0,X-MC,3.55,78.0,0.68,1.45,15.30,XY25,NTT,AC
53,NESTLEIND,1377.0,-0.04,12.86,7.56,21.40,22647.0,34141.0,299567.0,4.67,62.0,X-LC,7.94,12.0,1.51,2.10,20.36,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,2.03,-27.87,98.03,42.84,45966.0,-18116.0,46890.0,-52.62,40.0,X-SC,37.22,83.0,-0.39,0.33,2.03,XY24,NTT,MISC
7,ATULAUTO,844.00,1.78,-21.09,84.48,45.57,131409.0,-41582.0,155550.0,3364.86,43.0,M-SC,4.01,244.0,-0.32,1.09,10.28,XY24,NTT,AUTO
41,INFY,2275.00,1.51,6.77,45.05,54.87,150498.0,21181.0,334069.0,-16.73,64.0,X-LC,3.14,2.0,0.14,2.34,15.61,X40,BTT,IT
50,LTIM,7201.88,1.21,9.65,20.07,31.66,54171.0,23759.0,269910.0,5.43,65.0,H-LC,12.55,43.0,0.44,1.89,49.76,X200,ATH,IT
85,WIPRO,420.00,1.19,1.47,69.77,72.27,119960.0,2488.0,171936.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.20,7.73,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RAJOOENG,143.10,-3.61,-30.18,99.94,39.61,71527.0,-30930.0,71570.0,-52.76,20.0,H-SC,19.29,135.0,-0.43,0.50,0.00,AR,ATH,MISC
48,KPIGREEN,730.66,-3.13,-12.44,67.83,46.95,74415.0,-15589.0,109708.0,-19.22,26.0,H-SC,6.21,154.0,-0.21,0.77,37.44,MH,ATH,POWER
27,GLAXO,3466.20,-2.39,-2.12,42.16,39.15,82236.0,-4228.0,195056.0,-21.47,28.0,X-MC,9.07,60.0,-0.05,1.37,26.26,X40,ATH,PHARMA
65,SHALBY,327.00,-2.11,-18.66,53.80,25.10,79272.0,-33797.0,147346.0,982.93,29.0,M-SC,16.46,250.0,-0.43,1.03,18.08,XY24,NTT,HEALTHCARE
81,VALIANTORG,838.00,-1.99,-45.29,218.33,74.15,195416.0,-74100.0,89505.0,-260.95,26.0,H-SC,14.53,149.0,-0.38,0.63,14.73,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.97,-0.34,107.43,106.72,132044.0,-424.0,122912.0,-50.63,33.0,H-SC,4.08,139.0,-0.0,0.86,22.95,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.74,-4.69,118.15,107.91,166258.0,-6926.0,140718.0,-22.58,34.0,M-SC,10.34,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.57,-8.47,9.28,0.02,14826.0,-14791.0,159768.0,143.86,72.0,M-SC,5.48,234.0,-1.00,1.12,38.97,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.63,-13.69,15.97,0.09,13890.0,-13800.0,86977.0,101.92,52.0,M-SC,4.28,253.0,-0.99,0.61,14.27,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.17,-22.43,46.29,13.47,96824.0,-60498.0,209169.0,-68.12,27.0,H-SC,1.53,158.0,-0.62,1.46,6.36,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,1.03,-22.53,70.42,32.03,71923.0,-29701.0,102134.0,-16.20,51.0,H-SC,6.61,148.0,-0.41,0.71,14.40,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.19,1.47,69.77,72.27,119960.0,2488.0,171936.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.2,7.73,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.19,1.47,69.77,72.27,119960.0,2488.0,171936.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.20,7.73,XR,NTT,IT
38,INDIAMART,4810.62,-0.97,-0.34,107.43,106.72,132044.0,-424.0,122912.0,-50.63,33.0,H-SC,4.08,139.0,-0.00,0.86,22.95,AR,ATH,MISC
25,FINCABLES,1641.55,-0.74,-4.69,118.15,107.91,166258.0,-6926.0,140718.0,-22.58,34.0,M-SC,10.34,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.62,-1.93,38.30,35.63,78718.0,-4044.0,205530.0,-15.85,36.0,H-MC,4.52,119.0,-0.05,1.44,14.42,AR,ATH,PHARMA
37,IEX,219.00,0.30,-3.94,54.72,48.63,105108.0,-7871.0,192083.0,-36.00,54.0,H-SC,15.58,136.0,-0.07,1.34,7.16,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.25,-35.63,110.04,35.20,91142.0,-45844.0,82826.0,8.59,21.0,X-SC,12.65,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,-1.46,-10.40,28.60,15.23,51342.0,-20831.0,179516.0,-23.41,22.0,X-MC,6.53,66.0,-0.41,1.26,18.50,X40N,NTT,REALTY
76,TMPV,600.00,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
34,HONAUT,58357.33,0.31,-15.91,70.44,43.32,96475.0,-25912.0,136960.0,-27.69,25.0,X-SC,2.03,90.0,-0.27,0.96,5.16,X40N,ATH,ELECTRICAL
27,GLAXO,3466.20,-2.39,-2.12,42.16,39.15,82236.0,-4228.0,195056.0,-21.47,28.0,X-MC,9.07,60.0,-0.05,1.37,26.26,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.29,-0.64,11.15,10.44,22172.0,-1286.0,198852.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.89,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.08,1.19,29.42,30.97,52617.0,2109.0,178847.0,-22.64,48.0,X-MC,1.53,75.0,0.04,1.25,13.90,X40,ATH,INSURANCE
3,ACC,3906.0,-0.11,-23.17,113.66,64.15,207782.0,-55141.0,182810.0,-55.00,43.0,X-SC,1.89,82.0,-0.27,1.28,2.55,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.52,-15.18,47.97,25.51,75721.0,-28245.0,157850.0,-3.08,52.0,H-LC,1.92,49.0,-0.37,1.10,16.32,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.25,-35.63,110.04,35.20,91142.0,-45844.0,82826.0,8.59,21.0,X-SC,12.65,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.31,-13.95,72.53,48.45,125135.0,-27978.0,172528.0,-20.09,28.0,X-MC,7.49,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
76,TMPV,600.00,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
54,PAGEIND,51605.07,0.32,-4.70,32.29,26.07,50385.0,-7700.0,156040.0,-29.81,37.0,X-MC,8.85,55.0,-0.15,1.09,1.15,X40,ATH,APPARELS
55,PGHH,17907.65,0.76,-3.29,38.30,33.76,74386.0,-6600.0,194220.0,-31.84,39.0,X-MC,3.16,57.0,-0.09,1.36,1.74,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,2.03,-27.87,98.03,42.84,45966.0,-18116.0,46890.0,-52.62,40.0,X-SC,37.22,83.0,-0.39,0.33,2.03,XY24,NTT,MISC
60,RELAXO,1176.00,-0.94,-47.60,190.66,52.31,145026.0,-69095.0,76065.0,-44.25,28.0,X-SC,3.79,91.0,-0.48,0.53,1.82,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.25,-35.63,110.04,35.20,91142.0,-45844.0,82826.0,8.59,21.0,X-SC,12.65,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.17,2.62,20.91,24.08,27240.0,3324.0,130274.0,-5.65,44.0,X-SC,4.75,89.0,0.12,0.91,22.75,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.87,-4.73,27.62,21.58,36877.0,-6624.0,133515.0,-51.94,33.0,X-MC,7.02,56.0,-0.18,0.93,18.20,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.06,-14.45,39.42,19.28,116675.0,-49978.0,295978.0,-25.57,63.0,X-LC,7.64,1.0,-0.43,2.07,9.42,X40,ATH,IT
41,INFY,2275.00,1.51,6.77,45.05,54.87,150498.0,21181.0,334069.0,-16.73,64.0,X-LC,3.14,2.0,0.14,2.34,15.61,X40,BTT,IT
76,TMPV,600.00,-1.39,-15.36,67.97,42.18,159027.0,-42444.0,233966.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.19,-9.75,50.32,35.67,143441.0,-30784.0,285058.0,-20.46,40.0,X-LC,6.05,4.0,-0.21,2.00,2.76,X40N,ATH,FMCG
43,ITC,452.00,-0.29,-0.64,11.15,10.44,22172.0,-1286.0,198852.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.89,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.03,-36.86,110.45,32.89,53847.0,-28454.0,48752.0,-701.92,66.0,L-MC,5.64,259.0,-0.53,0.34,34.26,XR,NTT,BANKS
6,ASIANTILES,137.00,-1.08,-13.70,110.51,81.67,89469.0,-12850.0,80960.0,7277.78,49.0,L-SC,18.55,271.0,-0.14,0.57,56.78,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.86,-12.21,58.94,39.54,52346.0,-12347.0,88812.0,-31.30,68.0,M-SC,6.84,220.0,-0.24,0.62,25.23,AR,ATH,AUTO
51,MASFIN,398.61,-1.81,-4.68,28.05,22.05,26196.0,-4590.0,93390.0,-17.80,51.0,H-SC,7.23,164.0,-0.18,0.65,35.75,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.60,-20.94,55.62,23.03,78745.0,-37495.0,141576.0,53.51,46.0,H-SC,11.02,167.0,-0.48,0.99,40.47,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.57,-8.47,9.28,0.02,14826.0,-14791.0,159768.0,143.86,72.0,M-SC,5.48,234.0,-1.00,1.12,38.97,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,0.45,1.00,19.75,20.95,50221.0,2517.0,254285.0,-0.07,77.0,X-LC,12.76,31.0,0.05,1.78,36.99,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,1.03,-36.86,110.45,32.89,53847.0,-28454.0,48752.0,-701.92,66.0,L-MC,5.64,259.0,-0.53,0.34,34.26,XR,NTT,BANKS
50,LTIM,7201.88,1.21,9.65,20.07,31.66,54171.0,23759.0,269910.0,5.43,65.0,H-LC,12.55,43.0,0.44,1.89,49.76,X200,ATH,IT
0,5PAISA,593.00,-1.65,-35.80,75.86,12.90,106413.0,-78221.0,140275.0,106.73,57.0,H-SC,14.54,161.0,-0.74,0.98,16.64,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,44.74
2,50,75.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.51
MC    29.28
LC    25.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.91
X40      21.21
X40N     11.84
XR        9.61
XY25      9.13
AR        8.95
OX40N     7.69
X200      1.89
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    22.86
X-LC    20.04
M-SC    12.07
X-SC     8.16
H-MC     4.71
H-LC     2.99
L-SC     1.39
M-MC     1.37
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-4.80,38.70
IT,13.85,-14.14,73.06
FINANCE,9.43,-15.01,63.60
MISC,7.41,-22.67,72.93
PAINTS,6.01,-7.43,23.39
ELECTRICAL,5.88,-11.71,52.62
INSURANCE,4.40,-0.53,35.96
PHARMA,3.84,-1.98,34.62
AUTO,3.35,-20.15,71.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3049303.0,21
AR,1292633.0,10
XR,1288439.0,13
X40,1008040.0,14
X40N,758327.0,9
OX40N,692676.0,10
XY25,337103.0,6
SR,279107.0,2
MH,74415.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3562091.0,25
M-SC,1337264.0,15
X-MC,1217374.0,16
X-LC,901217.0,11
X-SC,754537.0,8
H-MC,405883.0,3
L-SC,262455.0,3
H-LC,129892.0,2
M-LC,119960.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218428.0      6
           AR         877562.0      5
           XR         790899.0      7
M-SC       XY24       782531.0      6
X-MC       X40        443777.0      7
X-LC       X40        386697.0      5
X-MC       XY24       347816.0      3
H-SC       OX40N      321680.0      4
X-SC       X40N       295983.0      3
M-SC       OX40N      282871.0      5
X-SC       XY24       280988.0      3
H-SC       SR         279107.0      2
X-MC       X40N       249638.0      4
X-LC       XY24       230548.0      2
H-MC       AR         216891.0      2
X-LC       X40N       212706.0      2
H-MC       XY24       188992.0      1
X-SC       X40        177566.0      2
X-MC       XY25       176143.0      2
L-SC       XR         174330.0      2
M-SC       XR         149403.0      2
           AR         122459.0      2
M-LC       XR         119960.0      1
L-SC       OX40N       88125.0      1
H-LC       AR          75721.0      1
H-SC       MH          74415.0      1
X-LC       XY25        71266.0      2
H-LC       X200        54171.0      1
L-MC       XR          53847.0      1
M-MC       XY25        49312.0      1
L-LC       XY25        40382.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
